# `GroupBy`, `agg` (aggregrate), 'orderBy' operations
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536
In this notebook, we will illustrate the `groupBy`, agggreate (`agg`), and `orderBy` operations that can be performed on a DataFrame object.<br>
We will also show the use of standard functions that can be imported. 

### [Spark dataframe guide](https://spark.apache.org/docs/2.2.0/sql-programming-guide.html)


### Creating SparkSession and build an app

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark1 = SparkSession.builder.appName('aggs').getOrCreate()

### Read a dataset from a CSV file into a dataframe

In [3]:
file_path='../datasets/sparkbyexamples/sales_info.csv'

In [4]:
df = spark1.read.csv(file_path,inferSchema=True,header=True)

### See how the data looks like and print the schema

In [5]:
df.show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



### `groupBy`

In [7]:
df.groupby('Company')

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

#### We can perform a variety of operations on this `pyspark.sql.group.GroupedData` object. 
For example, taking a `mean` (average).

In [8]:
df.groupby('Company').mean()

DataFrame[Company: string, avg(Sales): double]

So, the operation returns a DataFrame, not the actual mean. **Lazy evaluation** at play.

In [9]:
df.groupby('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



#### Some other operations
* `max`
* `min`
* `count`

In [10]:
df.groupby('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [15]:
df.groupby('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [ ]:
df.groupby('Company').count().show()

### `agg` or aggregrating
This method works with a slightly unconventional syntax. We have to pass on a dictionary as argument.<br>
Suppose, we want the sum total of sales. We use `sum` but pass it as the value of the key `Sales` of a dictionary, 

In [ ]:
df.agg({'Sales':'sum'}).show()

Suppose we want the maximum of all sales

In [ ]:
df.agg({'Sales':'max'}).show()

### Combine `groupBy` and `agg` to get info by company

In [16]:
group_data = df.groupBy('Company')

In [ ]:
group_data.agg({'Sales':'max'}).show()

### Import and use other functions

In [17]:
from pyspark.sql.functions import stddev,countDistinct,count,avg

In [ ]:
df.agg({'Sales':'stddev'}).show()

#### We could compute the same quantity using simple `select` too

In [18]:
df.select(stddev('Sales')).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



#### We can count how many total sales and how many of them are distinct in value

In [19]:
df.select(count('Sales')).show()

+------------+
|count(Sales)|
+------------+
|          12|
+------------+



In [22]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



### Formatting results
Many of these functions and mathematical operations produce results with long number format. We can change the format suitably using `format_number` function. We can also change the long column name to something shorter using `alias` method.

In [23]:
from pyspark.sql.functions import format_number

In [24]:
sales_std = df.select(stddev('Sales').alias('std'))

In [25]:
sales_std

DataFrame[std: double]

In [26]:
sales_std=sales_std.select(format_number('std',2).alias('final'))

In [27]:
sales_std

DataFrame[final: string]

In [28]:
sales_std.show()

+------+
| final|
+------+
|250.09|
+------+



### Use `orderBy` to order a particular column by value

In [ ]:
df.show()

In [11]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



#### To order by descending values, it is bit more involved. You have to pass on the actual column object (not just the name as a string) and call `desc` method on it.

In [12]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [13]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.read.option("header","true").csv("../datasets/online_retail/Online_Retail_Data_Set.csv")
from pyspark.sql.functions import col
df.show(5)
#select(col('InvoiceNo'))
#df.groupBy('Country').count().show()
# Group by 'Country', count the number of rows, and order by count descending
df.groupBy('Country').count()\
  .withColumnRenamed('count', 'NBR')\
  .orderBy(col('NBR').desc())\
  .show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 5 rows

+---------------+------+
|    